# Solution for NLP

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

import pandas as pd

## Speeches I

In [ ]:
from pickle import dump
from string import digits, punctuation

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
folder = Path("../data/speeches/")
corpus = []
for f in folder.glob("R0*"):
    try:
        text = f.read_text(encoding="utf8")
        corpus.append(text)
    except UnicodeDecodeError:
        print(f)

In [ ]:
_stopwords = nltk.corpus.stopwords.words('english')
_stemmer = nltk.snowball.SnowballStemmer('english')


def tokenize_and_stem(text):
    """Return tokens of text deprived of numbers and interpunctuation."""
    text = text.translate(str.maketrans({p: "" for p in digits + punctuation}))
    return [_stemmer.stem(t) for t in nltk.word_tokenize(text.lower())]


vectorizer = TfidfVectorizer(stop_words=_stopwords, tokenizer=tokenize_and_stem, ngram_range=(1, 3))
tfidf_matrix = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names()

In [ ]:
FNAME = Path("../output/speech_matrix.pk")
with open(FNAME, "wb") as ouf:
    dump(tfidf_matrix, ouf)

terms = pd.DataFrame(terms)
terms.columns = ["terms"]
terms.to_csv(Path('../output/terms.csv'))

## Speeches II

In [ ]:
from pickle import load

from scipy.cluster.hierarchy import linkage, dendrogram

In [ ]:
with open(Path("../output/speech_matrix.pk"), "rb") as inf:
    tfidf_matrix = load(inf)

In [ ]:
tfidf_matrix.shape

In [ ]:
linkage_matrix = linkage(tfidf_matrix.todense(), metric="cosine", method="complete")
plt.figure(figsize=(10, 5))
dendrogram(linkage_matrix)
plt.show()
fig.savefig(Path("../output/speeches_dendrogram.pdf"))

## Job Ads

In [ ]:
import locale

import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Read file into one list and strip whitespace characters at end
FNAME = Path("../data/Stellenanzeigen.txt")
with open(FNAME, "r", encoding="utf8") as inf:
    corpus = [x.rstrip() for x in inf]

In [ ]:
# Difficulties:
# a) Newspaper name and date in same line, ad text in next
# b) Some ads contain a title, thus some entries spend 2 lines and others 3 lines
# !! Therefore multiple ways !!

In [ ]:
# THE SPLIT WAY
# Idea: Join all entries on _same_ separator as that for newspaper and date, and then
# split on the double separator generated by the empty line between entries
SEP = ", "
temp = SEP.join(corpus).split(SEP*2)
paper, date, text = zip(*[e.split(SEP, 3) for e in temp])

assert(len(paper) == len(date) == len(text) == 110)

In [ ]:
# THE ITERATIVE WAY
# Idea: Iterate over each line and decide:
# If a line ends with a year, it's meta information (title and date), but
# if line is empty, conclude the entry, otherwise it's still text

paper = []
date = []
text = []
new_entry = []
for line in corpus:
    if line and line[-4:].isdigit():  # Line is header
        new_paper, new_date = line.rsplit(", ", 1)
        paper.append(new_paper)
        date.append(new_date)
    elif line:  # Line is other content
        new_entry.append(line)
    else:  # Line is empty
        new_entry = " ".join(new_entry)
        text.append(new_entry)
        new_entry = []
# Also add the last entry
text.append(" ".join(new_entry))

assert(len(paper) == len(date) == len(text) == 110)

In [ ]:
# Turn into DataFrame
df = pd.DataFrame({"newspaper": paper, "date": date, "text": text})
df.head()

In [ ]:
# Convert datetime
# Possibble difficulty: pandas' date parser only works 
# when the locale is correct (there are other ways, too)
locale.setlocale(locale.LC_ALL, 'de_CH.utf8')  # On Unix-systems use de_CH.utf8

df["date"] = pd.to_datetime(df["date"], format="%d. %B %Y")

In [ ]:
# Create column decade
df["decade"] = (df["date"].dt.year//10)*10  # // stands for integer division
print(df["decade"].value_counts())

In [ ]:
# Count number of words (not characters) for each text
df["length"] = df["text"].str.split().apply(len)  # One might want to clean first, but not mandatory

In [ ]:
df.head()

In [ ]:
df.head(10)["length"].mean()

In [ ]:
# PANDAS PLOT
# Aggregate by decade and then plot
grouped_dec = df.groupby("decade")["length"].mean()
fig, ax = plt.subplots()
grouped_dec.plot(legend=None, ax=ax)
ax.set(xlabel="Decade", ylabel="Average length")
plt.savefig(Path("../output/job_ads.pdf"))

In [ ]:
# SEABORN PLOT
fig, ax = plt.subplots()
sns.lineplot(data=df, x="decade", y="length", ax=ax)
ax.set(xlabel="Decade", ylabel="Average length")
plt.savefig(Path("../output/job_ads.pdf"))

In [ ]:
df.groupby("decade")["length"].mean()

In [ ]:
# Hint: pandas' sum() after groupby can easily concatenate text, but
# you need to add a blank beforehand
df["text"] += " "
grouped_corp = df.groupby("decade")["text"].sum()
print(grouped_corp.shape)

In [ ]:
# For simplicity use CountVectorizer
# Difficulty: There are English texts too, but you can only
# use one stemmer

remove = digits + punctuation + "‘’“”„§"

_stemmer = nltk.snowball.SnowballStemmer('german')
_stopwords = nltk.corpus.stopwords.words("german")
_stopwords.extend(["fur", "unt", "sowi"])
_stopwords.extend(nltk.corpus.stopwords.words("english"))

def tokenize_and_stem(text):
    """Return tokens of text deprived of numbers and interpunctuation."""
    text = text.translate(str.maketrans({p: "" for p in remove}))
    return [_stemmer.stem(t) for t in nltk.word_tokenize(text.lower())]

_stopwords = tokenize_and_stem(" ".join(_stopwords))
CV = CountVectorizer(stop_words=_stopwords, tokenizer=tokenize_and_stem)

In [ ]:
count_matrix = CV.fit_transform(grouped_corp.values).todense()
count_df = pd.DataFrame(count_matrix, index=grouped_corp.index,
                        columns=CV.get_feature_names())
count_df = count_df.T

In [ ]:
for decade in count_df.columns:
    terms = count_df.sort_values(decade).tail().index
    print(decade, ":", " - ".join(terms))